# Credit Card Fraud Detection::

In [36]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Download dataset from this link:

https://www.kaggle.com/mlg-ulb/creditcardfraud

# Description about dataset::

The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 


### Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

# WORKFLOW :

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Standardized the Input Variables. 

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

5.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

7.Train the Model with Epochs (100).

8.If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

9.Prediction should be > 92%
10.Evaluation Step
11Prediction


# Task::

## Identify fraudulent credit card transactions.

In [44]:
cd /content/drive/MyDrive/credit-card/creditcard.csv/

/content/drive/MyDrive/credit-card/creditcard.csv


1.Load **Data**

In [46]:
import numpy as np
import pandas as pd
import tensorflow as tf

df =pd.read_csv("/content/drive/MyDrive/credit-card/creditcard.csv/creditcard.csv")

In [47]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,4.356170,-1.593105,2.711941,-0.689256,4.626942,-0.924459,1.107641,1.991691,0.510632,-0.682920,1.475829,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,-0.975926,-0.150189,0.915802,1.214756,-0.675143,1.164931,-0.711757,-0.025693,-1.221179,-1.545556,0.059616,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,-0.484782,0.411614,0.063119,-0.183699,-0.510602,1.329284,0.140716,0.313502,0.395652,-0.577252,0.001396,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,-0.399126,-1.933849,-0.962886,-1.042082,0.449624,1.962563,-0.608577,0.509928,1.113981,2.897849,0.127434,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )**bold text**

In [48]:
df.isnull().any()

Time      False
V1        False
V2        False
V3        False
V4        False
V5        False
V6        False
V7        False
V8        False
V9        False
V10       False
V11       False
V12       False
V13       False
V14       False
V15       False
V16       False
V17       False
V18       False
V19       False
V20       False
V21       False
V22       False
V23       False
V24       False
V25       False
V26       False
V27       False
V28       False
Amount    False
Class     False
dtype: bool

In [49]:
np.where(df.applymap(lambda x: x=='') )

(array([], dtype=int64), array([], dtype=int64))

In [50]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,1.768627e-15,9.170318e-16,-1.810658e-15,1.693438e-15,1.479045e-15,3.482336e-15,1.392007e-15,-7.528491e-16,4.328772e-16,9.049732e-16,5.085503e-16,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,1.088850e+00,1.020713e+00,9.992014e-01,9.952742e-01,9.585956e-01,9.153160e-01,8.762529e-01,8.493371e-01,8.381762e-01,8.140405e-01,7.709250e-01,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,-2.458826e+01,-4.797473e+00,-1.868371e+01,-5.791881e+00,-1.921433e+01,-4.498945e+00,-1.412985e+01,-2.516280e+01,-9.498746e+00,-7.213527e+00,-5.449772e+01,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,-5.354257e-01,-7.624942e-01,-4.055715e-01,-6.485393e-01,-4.255740e-01,-5.828843e-01,-4.680368e-01,-4.837483e-01,-4.988498e-01,-4.562989e-01,-2.117214e-01,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,-9.291738e-02,-3.275735e-02,1.400326e-01,-1.356806e-02,5.060132e-02,4.807155e-02,6.641332e-02,-6.567575e-02,-3.636312e-03,3.734823e-03,-6.248109e-02,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,4.539234e-01,7.395934e-01,6.182380e-01,6.625050e-01,4.931498e-01,6.488208e-01,5.232963e-01,3.996750e-01,5.008067e-01,4.589494e-01,1.330408e-01,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,2.374514e+01,1.201891e+01,7.848392e+00,7.126883e+00,1.052677e+01,8.877742e+00,1.731511e+01,9.253526e+00,5.041069e+00,5.591971e+00,3.942090e+01,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [51]:
non_fraud = df[df['Class']==0]
fraud = df[df['Class'] == 1]

non_fraud = non_fraud.sample(fraud.shape[0])
non_fraud.shape
data= fraud.append(non_fraud, ignore_index=True)
data.shape
data['Class'].value_counts()
x_data = data.drop(columns = 'Class', axis=0)
label = data['Class']

In [52]:
non_fraud.shape, fraud.shape

((492, 31), (492, 31))

In [53]:
x_data.shape

(984, 30)

In [54]:
x_data.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
      dtype='object')

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels). Validation will be taken later during model compilation


In [55]:
from sklearn.model_selection import train_test_split


In [56]:
train_data, test_data, train_labels, test_labels = train_test_split(x_data, label, test_size = .3, random_state=1, stratify= label)

In [57]:
test_data

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
902,141463.0,-1.776458,-7.163714,-4.862215,1.098859,-1.813468,-0.506299,3.087924,-0.935254,-1.329562,0.297040,0.070899,-0.627380,-1.655158,1.740285,-0.531239,-1.670406,0.032309,1.804050,-0.948660,3.468990,1.160876,-0.846714,-1.949561,-0.340665,-0.318212,0.815517,-0.528004,0.250086,2083.20
747,63568.0,1.245674,0.166975,0.488306,0.635322,-0.562777,-1.011073,0.014953,-0.160211,0.170362,-0.044575,-0.356749,-0.073460,-0.517760,0.406969,1.124147,0.342470,-0.374656,-0.438992,-0.116091,-0.132080,-0.262581,-0.816264,0.140304,0.357827,0.186423,0.096544,-0.035866,0.018495,8.99
701,133259.0,1.983251,-0.026721,-0.789903,0.198996,0.105802,-0.285375,-0.052923,-0.081532,0.397436,-0.035559,1.294300,2.162784,1.652150,0.068391,-0.472655,0.063703,-0.756693,-0.420161,0.483912,-0.094944,-0.410810,-1.069330,0.502782,0.726285,-0.433275,-0.931290,0.011070,-0.035761,4.73
827,126642.0,2.146681,-0.952284,-1.007156,-1.115433,-0.527976,-0.086400,-0.854794,0.111446,-0.256762,0.906632,0.551777,-0.406065,-0.997255,0.151250,-0.451153,1.387642,-0.056497,-1.217023,1.217164,-0.052446,-0.253879,-0.897440,0.468308,0.201607,-0.552647,-0.697317,-0.023476,-0.052765,19.99
358,118532.0,-5.961457,5.313382,-6.674320,6.028975,-1.387560,0.670638,-4.128987,-4.765894,-1.005259,0.453505,4.431736,-5.142737,-1.181692,-8.755449,1.004356,-2.065026,-3.261435,0.125223,0.936966,-1.116581,5.556642,-1.501808,1.355172,0.141093,0.077913,0.473988,0.287129,1.468653,105.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,100739.0,1.902854,0.593680,-0.456897,3.697796,0.802485,0.926333,-0.199434,0.043918,0.158930,1.193617,1.150583,-1.583013,2.913999,1.410842,-2.184982,1.190938,-0.602010,0.557218,-1.371540,-0.191998,0.008237,0.330587,0.107412,0.229523,0.012114,-0.034893,-0.046621,-0.053418,9.46
326,96291.0,-3.552173,5.426461,-3.731810,6.679062,-2.187543,2.433940,-8.748110,-12.108284,-2.856359,-5.665862,2.540836,-6.147054,1.796891,-4.328989,-0.150037,-1.839005,-3.354638,-0.282307,0.148592,3.792667,-5.688990,2.510980,0.953933,-0.542506,-0.620152,0.406013,0.023025,0.164741,33.59
578,59491.0,1.574053,-1.141845,0.222343,-1.635912,-1.160860,0.004103,-1.259222,-0.060747,-1.519154,1.420082,-1.746858,-1.305870,0.952733,-0.602868,0.931164,0.254631,-0.141763,0.751778,0.081328,-0.241818,-0.143975,-0.013897,-0.185193,-1.008310,0.527020,-0.018600,0.048531,0.015522,24.30
934,65369.0,-0.782347,-0.562991,0.239272,-1.875068,1.240592,4.237554,0.405697,1.040642,0.497130,-1.244773,-0.861613,0.040083,-0.212297,-0.512841,-0.822973,0.424273,-0.659518,0.002973,0.176477,0.510723,-0.028753,-0.451392,0.505361,1.034185,-0.367765,0.723026,0.003932,0.154045,245.55


In [58]:
ww = np.where(df.Amount.apply(lambda x: x == 0))
(df['Amount'].values == 0 ).sum()

1825

In [59]:
train_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
485,166883.0,2.091900,-0.757459,-1.192258,-0.755458,-0.620324,-0.322077,-1.082511,0.117200,-0.140927,0.249311,1.338318,-0.329759,-0.363591,-1.824839,-0.229679,1.842347,1.194212,0.037467,0.423099,0.037438,0.288253,0.831939,0.142007,0.592615,-0.196143,-0.136676,0.020182,-0.015470,19.95
812,39047.0,-1.168450,0.233458,1.110144,-3.889895,-0.946954,-1.247170,-0.108759,0.291452,-0.038435,-1.174980,-1.095943,0.291773,0.711999,-0.071909,0.529178,-1.058400,-0.631254,1.632254,-1.683249,-0.561280,-0.137057,0.039133,-0.238275,0.043739,0.302954,-0.199911,0.075377,0.092215,15.21
58,25231.0,-16.598665,10.541751,-19.818982,6.017295,-13.025901,-4.128779,-14.118865,11.161144,-4.099551,-9.222826,6.329365,-8.952191,-0.138364,-9.825054,0.057224,-7.541687,-14.259599,-5.035052,1.432268,1.534920,1.725853,-1.151606,-0.680052,0.108176,1.066878,-0.233720,1.707521,0.511423,99.99
115,41170.0,-6.498086,4.750515,-8.966558,7.098854,-6.958376,-2.822126,-10.333406,4.031907,-6.648778,-11.634414,6.877571,-13.697686,0.463040,-13.044182,-0.309229,-12.317580,-24.019099,-9.335193,1.951890,0.568338,2.158143,0.111510,0.216414,0.584661,0.760360,0.081972,1.415068,0.035124,83.38
763,139550.0,1.991610,-0.071493,-1.805354,0.615366,0.181878,-0.683972,-0.119584,-0.014290,0.804568,-0.470332,0.632746,-0.062401,-1.647727,-1.228613,-0.895616,0.414904,1.010349,0.865909,0.227012,-0.250434,-0.009596,0.183591,0.003774,-0.503070,0.038437,0.648955,-0.052131,-0.046350,9.99


In [60]:
test_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
902,141463.0,-1.776458,-7.163714,-4.862215,1.098859,-1.813468,-0.506299,3.087924,-0.935254,-1.329562,0.297040,0.070899,-0.627380,-1.655158,1.740285,-0.531239,-1.670406,0.032309,1.804050,-0.948660,3.468990,1.160876,-0.846714,-1.949561,-0.340665,-0.318212,0.815517,-0.528004,0.250086,2083.20
747,63568.0,1.245674,0.166975,0.488306,0.635322,-0.562777,-1.011073,0.014953,-0.160211,0.170362,-0.044575,-0.356749,-0.073460,-0.517760,0.406969,1.124147,0.342470,-0.374656,-0.438992,-0.116091,-0.132080,-0.262581,-0.816264,0.140304,0.357827,0.186423,0.096544,-0.035866,0.018495,8.99
701,133259.0,1.983251,-0.026721,-0.789903,0.198996,0.105802,-0.285375,-0.052923,-0.081532,0.397436,-0.035559,1.294300,2.162784,1.652150,0.068391,-0.472655,0.063703,-0.756693,-0.420161,0.483912,-0.094944,-0.410810,-1.069330,0.502782,0.726285,-0.433275,-0.931290,0.011070,-0.035761,4.73
827,126642.0,2.146681,-0.952284,-1.007156,-1.115433,-0.527976,-0.086400,-0.854794,0.111446,-0.256762,0.906632,0.551777,-0.406065,-0.997255,0.151250,-0.451153,1.387642,-0.056497,-1.217023,1.217164,-0.052446,-0.253879,-0.897440,0.468308,0.201607,-0.552647,-0.697317,-0.023476,-0.052765,19.99
358,118532.0,-5.961457,5.313382,-6.674320,6.028975,-1.387560,0.670638,-4.128987,-4.765894,-1.005259,0.453505,4.431736,-5.142737,-1.181692,-8.755449,1.004356,-2.065026,-3.261435,0.125223,0.936966,-1.116581,5.556642,-1.501808,1.355172,0.141093,0.077913,0.473988,0.287129,1.468653,105.89


**3.Standardized the Input Variables. **

In [61]:
mean =  train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /=std
test_data -=mean
test_data /=std

In [62]:
test_data.dtypes

Time      float64
V1        float64
V2        float64
V3        float64
V4        float64
V5        float64
V6        float64
V7        float64
V8        float64
V9        float64
V10       float64
V11       float64
V12       float64
V13       float64
V14       float64
V15       float64
V16       float64
V17       float64
V18       float64
V19       float64
V20       float64
V21       float64
V22       float64
V23       float64
V24       float64
V25       float64
V26       float64
V27       float64
V28       float64
Amount    float64
dtype: object


6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)


In [63]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(10, activation= 'relu',input_shape=(train_data.shape[1],)))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(8, activation= 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(6, activation= 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation= 'sigmoid'))

In [64]:
from tensorflow.keras import models
from tensorflow.keras import layers

model1 = models.Sequential()
model1.add(layers.Dense(10, activation= 'tanh',input_shape=(train_data.shape[1],)))
model1.add(layers.Dropout(0.3))
model1.add(layers.Dense(8, activation= 'tanh'))
model1.add(layers.Dropout(0.3))
model1.add(layers.Dense(6, activation= 'tanh'))
model1.add(layers.Dropout(0.3))
model1.add(layers.Dense(1, activation= 'sigmoid'))

In [65]:
model.compile(loss ='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

7.Train the Model with Epochs (100).

In [66]:
model.fit(train_data, train_labels, epochs=100, validation_split= 0.3)

Epoch 1/100
16/16 [==============================] - 1s 25ms/step - loss: 0.7421 - accuracy: 0.4849 - val_loss: 0.6905 - val_accuracy: 0.5942
Epoch 2/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6960 - accuracy: 0.5569 - val_loss: 0.6815 - val_accuracy: 0.6570
Epoch 3/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6829 - accuracy: 0.5603 - val_loss: 0.6775 - val_accuracy: 0.7246
Epoch 4/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6824 - accuracy: 0.5815 - val_loss: 0.6725 - val_accuracy: 0.7585
Epoch 5/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6774 - accuracy: 0.6176 - val_loss: 0.6635 - val_accuracy: 0.7874
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.6665 - accuracy: 0.6542 - val_loss: 0.6523 - val_accuracy: 0.8116
Epoch 7/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6601 - accuracy: 0.6307 - val_loss: 0.6389 - val_accuracy: 0.8309
Epoch 8/100


In [67]:
model1.compile(loss ='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [68]:
model1.fit(train_data, train_labels, epochs=100, validation_split= 0.3)

Epoch 1/100
16/16 [==============================] - 1s 21ms/step - loss: 0.9152 - accuracy: 0.3965 - val_loss: 0.6691 - val_accuracy: 0.5942
Epoch 2/100
16/16 [==============================] - 0s 6ms/step - loss: 0.7174 - accuracy: 0.5911 - val_loss: 0.5552 - val_accuracy: 0.7488
Epoch 3/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6241 - accuracy: 0.6619 - val_loss: 0.4801 - val_accuracy: 0.8068
Epoch 4/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5777 - accuracy: 0.7128 - val_loss: 0.4353 - val_accuracy: 0.8454
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5185 - accuracy: 0.7553 - val_loss: 0.4035 - val_accuracy: 0.8792
Epoch 6/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5161 - accuracy: 0.7952 - val_loss: 0.3810 - val_accuracy: 0.8792
Epoch 7/100
16/16 [==============================] - 0s 6ms/step - loss: 0.5018 - accuracy: 0.7920 - val_loss: 0.3658 - val_accuracy: 0.8841
Epoch 8/100


9.Prediction should be > 92%


In [69]:
test_loss_Score1, test_acc_score1 = model1.evaluate(test_data, test_labels)

10/10 [==============================] - 0s 2ms/step - loss: 0.1765 - accuracy: 0.9493


In [70]:
test_loss_Score, test_acc_score = model.evaluate(test_data, test_labels)

10/10 [==============================] - 0s 2ms/step - loss: 0.1787 - accuracy: 0.9527


In [71]:
test_acc_score*100

95.27027010917664

10.Evaluation Step 11Prediction


In [72]:
model.predict(np.array(test_data.iloc[1,:]).reshape(1,test_data.shape[1]))

array([[0.07090822]], dtype=float32)

In [73]:
prediction = model.predict(test_data).astype(dtype='u8')


In [74]:
prediction[:15]

array([[0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]], dtype=uint64)

In [75]:
test_labels[:15]

902    0
747    0
701    0
827    0
358    1
794    0
273    1
824    0
11     1
204    1
897    0
887    0
949    0
882    0
851    0
Name: Class, dtype: int64